In [33]:
import pandas as pd
import pyodbc
from sqlpush.sqlstd import *
from sqlpush.config import *

In [49]:
# how to fuzzy match : 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
fuzz.token_sort_ratio("Catherine Gitau M.", "Gitau Catherine")


94

In [35]:
# Prepare connection
cursor, conn = connectdb(config)


# Get DB columns names
tablecol = getcolumnnames(cursor, "belgium_addresses", "dbo")

In [41]:
def getdata(cursor):
    query = "SELECT * FROM belgium_addresses "
    cursor.execute(query)
    tablecol = [i[0] for i in cursor.fetchall()]
    return tablecol

In [42]:
# get to know who to do all those sql quetires
# fuzzy matching
#implement lisa's algorithm